### 轉換產品分類 Excel 成為階層式 JSON 架構

In [8]:
import pandas as pd
import json
categories_df = pd.read_excel("sources/設研院114年產品分類.xlsx")

In [10]:
categories_json = {}

categories_json['其他'] = {"其他": {"其他": "其他"}}
for domain in categories_df['前端_domain'].unique():
    subset_domain = categories_df[categories_df["前端_domain"] == domain]
    categories_json[domain] = {key: {} for key in subset_domain['前端_category'].unique()}
    categories_json[domain]['其他'] = {"其他": "其他"}

    for category in subset_domain['前端_category'].unique():
        subset_category = subset_domain[subset_domain['前端_category'] == category]
        categories_json[domain][category] = {key: {} for key in subset_category['前端_subcategory'].unique()}
        categories_json[domain][category]['其他'] = "其他"

        for subcategory in subset_category['前端_subcategory'].unique():
            subset_subcategory = subset_category[subset_category['前端_subcategory'] == subcategory]
            categories_json[domain][category][subcategory] = [key for key in  subset_subcategory['前端_further_subcategory'].unique()]
with open("utils/categories.json", "w") as f:
    json.dump(categories_json, f, ensure_ascii = False, indent = 4)

### 小範圍測試

In [1]:
from utils.llm_manager import LlmManager
from utils.prompt_manager import PromptManager
import pandas as pd
import json

model = LlmManager(chart = "products")
promptManager = PromptManager()
products_df = pd.read_excel("sources/products_momo_0604_classified.xlsx")

In [2]:
verified_data = []

for _, row in products_df.iloc[500:501, :].iterrows():

    input = promptManager.get_user_in_message(
        row['title'],
        row['domain'],
        row['category'],
        row['subcategory'],
        row['further_subcategory']
    )
    
    
    output_text = model.api_call(input)
    output_json = LlmManager.find_json_object(output_text)

    output_json["source_product_id"] = row['source_product_id']

    verified_data.append(output_json)

verified_df = pd.DataFrame(verified_data)

In [3]:
verified_df

,is_correct,suggested_domain,suggested_category,suggested_subcategory,suggested_further_subcategory,confidence,reasoning,source_product_id
0,False,智慧移動,機車用品,機車配件,TBD,0.8,The product is an 'air intake protection net' ...,momo_12487548


In [1]:
from utils.sheet_manager import SheetManager

SheetManager = SheetManager()